<div style='float: right'><img src='pic/fillomino.png'/></div>
## <div id='fillomino' />フィルオミノ

In [ ]:
import numpy as np
from itertools import product
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = """\
.2.4.2.
1.2.6.6
3..3..3
...5...
3..2..3
3.2.4.2
.3.3.1.""".split('\n')
n = len(data)

### 問題
* 盤面のすべてのマスに数字を入れます
* 同じ数字はその数字に等しい個数だけつながります
* つながった同じ数字は、別の同じ数字のつながったものと辺を共有しません

### 変数
* 略

### 制約
* 略

---

In [ ]:
dirs = ((-1,0),(0,-1),(0,1),(1,0))
def chk(rl, pre):
    for x, y in pre:
        for dx, dy in dirs:
            sx, sy = x+dx, y+dy
            if [sx,sy] not in pre:
                nw = np.array(sorted(pre + [[sx,sy]]))
                nw = (nw-nw.min(0)).tolist()
                if nw not in rl:
                    rl.append(nw)
candslist = [[np.array([(0,0)])]]
for rem in range(2,7):
    rl = []
    for pre in candslist[rem-2]:
        chk(rl, pre.tolist())
    candslist.append([np.array(cands) for cands in rl])
allcands = [cand for cands in candslist for cand in cands]
cands = []
lsts = [[] for _ in range(n*n)]
lsbys = [[] for _ in range(6)]
for i, j in product(range(n), range(n)):
    cnds = candslist[int(data[i][j])-1] if data[i][j].isdigit() else allcands
    for cnd in cnds:
        cl = str(len(cnd))
        for x, y in cnd:
            sftcnd = cnd-(x,y)+(i,j)
            xx, yy = sftcnd.T
            if 0<=xx.min() and xx.max()<n and 0<=yy.min() and yy.max()<n:
                for p, q in sftcnd:
                    if data[p][q] != '.' and data[p][q] != cl:
                        break
                else:
                    sftcnd = [tuple(xy) for xy in sftcnd]
                    if sftcnd not in cands:
                        t = len(cands)
                        cands.append(sftcnd)
                        for p, q in sftcnd:
                            lsts[p*n+q].append(t)
                        lsbys[len(sftcnd)-1].append(t)
m = LpProblem()
var = addbinvars(len(cands))
for lst in lsts:
    m += lpSum(var[i] for i in lst) == 1
for lsby in lsbys:
    nl = len(lsby)
    for i in range(nl):
        s = set((x+dx, y+dy) for x, y in cands[lsby[i]] for dx, dy in dirs)
        for j in range(i+1, nl):
            for k in cands[lsby[j]]:
                if k in s:
                    m += var[lsby[i]] + var[lsby[j]] <= 1
                    break
%time m.solve()
print(m.status)
res = [[0]*n for _ in range(n)]
for i, b in enumerate(np.vectorize(value)(var)):
    if b:
        for x, y in cands[i]:
            res[x][y] = len(cands[i])
res